# SIMD: The parallelism that can (sometimes) happen automatically

SIMD: Single-instruction, multiple data

(Also confusingly called vectorization)

## The architecture

Instead of computing four sums sequentially:

\begin{align}
x_1 + y_1 &\rightarrow z_1 \\
x_2 + y_2 &\rightarrow z_2 \\
x_3 + y_3 &\rightarrow z_3 \\
x_4 + y_4 &\rightarrow z_4
\end{align}

Modern processors have vector processing units that can do it all at once:

$$
\left(\begin{array}{cc}
x_1 \\
x_2 \\
x_3 \\
x_4
\end{array}\right)
+
\left(\begin{array}{cc}
y_1 \\
y_2 \\
y_3 \\
y_4
\end{array}\right)
\rightarrow
\left(\begin{array}{cc}
z_1 \\
z_2 \\
z_3 \\
z_4
\end{array}\right)
$$

## Making it happen

Simple task: compute the sum of a vector:

In [1]:
A = rand(100_000)
function simplesum(A)
    result = zero(eltype(A))
    for i in eachindex(A)
        @inbounds result += A[i]
    end
    return result
end

simplesum(A)

49987.295795696285

In [2]:
using BenchmarkTools
@btime simplesum($A)

  71.600 μs (0 allocations: 0 bytes)


49987.295795696285

So, is that good?

In [3]:
@btime sum($A)

  9.100 μs (0 allocations: 0 bytes)


49987.2957956958

We're slower that the builtin `sum` — and we're getting a different answer, too! Let's look at what happens with a 32-bit float instead of a 64 bit one. Each element has half the number of bits, so lets also double the length (so the total number of bits processed remains constant).

In [4]:
A32 = rand(Float32, length(A)*2)
@btime simplesum($A32)
@btime sum($A32);

  143.100 μs (0 allocations: 0 bytes)
  9.900 μs (0 allocations: 0 bytes)


That's even worse! What's going on here?  We're seeing an even multiple number
difference in our performance — perhaps Julia's builtin sum is using some
parallelism? Let's try using SIMD ourselves:

In [5]:
function simdsum(A)
    result = zero(eltype(A))
    @simd for i in eachindex(A)
        @inbounds result += A[i]
    end
    return result
end
@btime simdsum($A)
@btime simdsum($A32)

  8.333 μs (0 allocations: 0 bytes)
  8.233 μs (0 allocations: 0 bytes)


100023.03f0

What did that do and why don't we always use `@simd for` — or why doesn't Julia
just always use `@simd` for every `for` loop automatically?  Look at the values:

In [6]:
simplesum(A), simdsum(A), sum(A)

(49987.295795696285, 49987.29579569581, 49987.2957956958)

In [7]:
simplesum(A32), simdsum(A32), sum(A32)

(100022.11f0, 100023.03f0, 100023.05f0)

Why aren't they the same?

Without `@simd`, Julia is doing _exactly_ what we told it to do: it's taking
each element of our array and adding it to a big pile sequentially. Our answer
is smaller than what Julia's builtin `sum` thinks it is: that's because as our
pile gets bigger we begin losing the lower bits of each element that we're
adding, and those small losses begin to add up!

The `@simd` macro tells Julia that it can re-arrange floating point additions —
even if it would change the answer. Depending on your CPU, this may lead to 2x or 4x
or even 8x parallelism. Essentially, Julia is computing independent sums for
the even indices and the odd indices simultaneously:

\begin{align}
odds &\leftarrow 0 \\
evens &\leftarrow 0 \\
\text{loop}&\ \text{odd}\ i: \\
    &\left(\begin{array}{cc}
odds \\
evens
\end{array}\right)
\leftarrow
\left(\begin{array}{cc}
odds \\
evens
\end{array}\right)
+
\left(\begin{array}{cc}
x_{i} \\
x_{i+1}
\end{array}\right) \\
total &\leftarrow evens + odds
\end{align}

In many cases, Julia can and does know that a for-loop can be SIMD-ed and it
will take advantage of this by default!

In [8]:
B = rand(1:10, 100_000)
@btime simplesum($B)
@btime sum($B)
B32 = rand(Int32(1):Int32(10), length(B)*2)
@btime simplesum($B32)
@btime simdsum($B32)

  8.300 μs (0 allocations: 0 bytes)
  8.933 μs (0 allocations: 0 bytes)
  8.200 μs (0 allocations: 0 bytes)
  8.167 μs (0 allocations: 0 bytes)


1099928

How can we see if something is getting vectorized?

In [ ]:
@code_llvm simdsum(A32)

So what are the challenges?

* Biggest hurdle is that you have to convince Julia and LLVM that it's able to
  use SIMD instructions for your given algorithm. That's not always possible.
* There are lots of limitations of what can and cannot be SIMD-ed:

In [9]:
@doc @simd

```
@simd
```

Annotate a `for` loop to allow the compiler to take extra liberties to allow loop re-ordering

!!! warning
    This feature is experimental and could change or disappear in future versions of Julia. Incorrect use of the `@simd` macro may cause unexpected results.


The object iterated over in a `@simd for` loop should be a one-dimensional range. By using `@simd`, you are asserting several properties of the loop:

  * It is safe to execute iterations in arbitrary or overlapping order, with special consideration for reduction variables.
  * Floating-point operations on reduction variables can be reordered, possibly causing different results than without `@simd`.

In many cases, Julia is able to automatically vectorize inner for loops without the use of `@simd`. Using `@simd` gives the compiler a little extra leeway to make it possible in more situations. In either case, your inner loop should have the following properties to allow vectorization:

  * The loop must be an innermost loop
  * The loop body must be straight-line code. Therefore, [`@inbounds`](@ref) is   currently needed for all array accesses. The compiler can sometimes turn   short `&&`, `||`, and `?:` expressions into straight-line code if it is safe   to evaluate all operands unconditionally. Consider using the [`ifelse`](@ref)   function instead of `?:` in the loop if it is safe to do so.
  * Accesses must have a stride pattern and cannot be "gathers" (random-index   reads) or "scatters" (random-index writes).
  * The stride should be unit stride.

!!! note
    The `@simd` does not assert by default that the loop is completely free of loop-carried memory dependencies, which is an assumption that can easily be violated in generic code. If you are writing non-generic code, you can use `@simd ivdep for ... end` to also assert that:


  * There exists no loop-carried memory dependencies
  * No iteration ever waits on a previous iteration to make forward progress.


* You do need to think through the consequences of re-ordering your algorithm.

## A slightly trickier case

In [ ]:
using BenchmarkTools

In [10]:
function diff!(A, B)
    A[1] = B[1]
    for i in 2:length(A)
        @inbounds A[i] = B[i] - B[i-1]
    end
    return A
end
A = zeros(Float32, 100_000)
B = rand(Float32, 100_000)

diff!(A, B)
[B[1];diff(B)] == A

true

In [11]:
@btime diff!($A, $B)
@btime diff($B);

  7.900 μs (0 allocations: 0 bytes)
  25.300 μs (2 allocations: 390.70 KiB)


But what happens if we do it in-place?

In [12]:
Bcopy = copy(B)
@btime diff!($Bcopy, $Bcopy);

  262.400 μs (0 allocations: 0 bytes)


What happened?

In [ ]:
@code_llvm diff!(A, B)

We can manually assert that arrays don't alias (or have any loop-dependencies),
with the very special `@simd ivdep` flag, but this can be disastrous:

In [13]:
function unsafe_diff!(A, B)
    A[1] = B[1]
    @simd ivdep for i in 2:length(A)
        @inbounds A[i] = B[i] - B[i-1]
    end
    return A
end
@btime unsafe_diff!($A, $B)
[B[1];diff(B)] == A
Bcopy = copy(B)
unsafe_diff!(Bcopy, Bcopy)
[B[1];diff(B)] == Bcopy

  7.800 μs (0 allocations: 0 bytes)


false

If you really want to get your hands dirty, you can use the [SIMD.jl](https://github.com/eschnett/SIMD.jl)
package to manually specify those `<8 x float>` things that LLVM generates.
BUT: this is tricky and a pain; often it's just to be aware of what makes
Julia code automatically SIMD-able, some of the cases where it may fail, and
how to check its work.

## SIMD

* Exploits built-in parallelism in a processor
* Best for small, tight innermost loops
* Often happens automatically if you're careful
    * Follow the [perforance best practices](https://docs.julialang.org/en/v1/manual/performance-tips/)
    * `@inbounds` any array acesses
    * No branches or (non-inlined) function calls
* Can use `@simd` to allow Julia to break some rules to make it happen
    * But be careful, especially with `@simd ivdep`!
* Depending on processor and types involved, can yield 2-16x gains with extraordinarily little overhead
    * Smaller datatypes can improve this further; use `Float32` instead of `Float64`
      if possible, `Int32` instead of `Int64`, etc.
    * When buying a new processor, look for [AVX-512](https://en.wikichip.org/wiki/x86/avx-512) support